In [ ]:
import pandas as pd
import datetime
import time
import pickle
import os
import numpy as np
import vitaldb

df_match = pd.read_csv('ftn+_age_match_20201224-Copy1.csv',sep=',')
df_match['NRS_time']=pd.to_datetime(df_match['NRS_time'])
df_match = df_match.rename(columns={'Unnamed: 0':'index'})
#df_match.reset_index(inplace=True, drop=True)
'''vital -> csv
df_match['path']=[f_name.split('.')[0]+".csv" for f_name in df_match['path'].values.flatten()] '''

save_path = '../../../cranberry2/Preprocessing/vital_data/NIBP_250Hz_preop_5min/'
if not os.path.exists(save_path[:-1]):
    os.mkdir(save_path[:-1])
    
srate= 250
for i in range(1144,1145):
    filename = str(df_match.loc[i,'Value'])+','+str(df_match.loc[i,'index'])+','+ df_match.loc[i,'path']
    if not os.path.exists(save_path+filename):
        print(i,'...', end=' ')
        rec_path = '../../../cranberry2/Preprocessing/vital_data/pacu_full_vital/'+df_match.loc[i,'path']    
        vfile = vitaldb.vital_recs_time(rec_path,['Solar8000/NIBP_MBP'],interval=0.004).flatten()
        
        if len(vfile[0])==0:
            print('index ',i,' OPID ',df_match.loc[i,'opid'],'empty NIBP')
        else:
            Abs_time = [vfile[1]+datetime.timedelta(hours=9)+datetime.timedelta(seconds=j/srate) for j in range(len(vfile[0]))]
            dics = {'Abs_time':Abs_time,'NIBP':vfile[0]}
            df_NIBP = pd.DataFrame(data=dics)
            end_idx_list = df_NIBP[(df_NIBP['Abs_time'] > df_match.loc[i,'NRS_time'] - datetime.timedelta(seconds=1)) & (df_NIBP['Abs_time'] < df_match.loc[i,'NRS_time'] + datetime.timedelta(seconds=1))].index.tolist()

            if len(end_idx_list)==0:
                print('index ',i,' OPID ',df_match.loc[i,'opid']," no vital data at NRS time")                  
            else:
                end_idx=end_idx_list[0]
                start_idx= max(0,end_idx - 5 * 60 * srate)
                extraction = df_NIBP[start_idx:end_idx]
                with open(save_path+filename,'wb') as f:
                    pickle.dump(extraction, f)
                    print('done')

#이건 2분보다 짧아도 저장함

1144 ... 